![alt text](images/uspas.png)
# VUV and X-ray Free Electron Lasers
# Running Genesis with lume-genesis
#### In this session, we will use lume-genesis to do the following:
- load input files
- change input files
- run a genesis simulation
- load output results
- plot output results

Some comments and parameter descriptions taken from the Genesis manual: http://genesis.web.psi.ch/download.html   
Parts of this notebook taken from lume-genesis examples here: https://github.com/slaclab/lume-genesis/tree/master/examples  

##### Instructors: D. Nguyen, P. Anisimov, N. Neveu
##### Teaching Assistant: Y.S. Li
----------

In [ ]:
# importing python packages we will use
import os, h5py
import matplotlib.pyplot as plt
import numpy as np

# from lume-genesis
from genesis import Genesis
from genesis import parsers, lattice
from genesis.parsers import parse_beam_file
from genesis.writers import write_beam_file

----

# 1. Make a run directory

In [ ]:
os.getcwd() # your simulation will run here now

In [ ]:
# lume-genesis will create and use a temporary directory unless a working directory is specified.
# I prefer to make a new run directory in the current working directory
rundir = os.getcwd()+'/test_run' 
if not os.path.exists(rundir):
    os.makedirs(rundir)

# 2. Initialize lume-genesis

In [ ]:
# Initialize the run, make genesis object with some template input file
gen = Genesis('/home/vagrant/jupyter/FEL2021/examples/lcls_sase.in', verbose=True, workdir=rundir, use_tempdir=False)
# If use_tempdir=True, your files will run in a ~/tmp folder that is not in or visible from the working directory

The Genesis() call above loaded a premade genesis input and lattice file.   
After running this command, we can now look at what is in the input file and make changes in Python.   
All values in 'gen.input' are parameters in the input file. 
Take a look at the information from the input file in the following cells:

# 3. Look at loaded input with Python

In [ ]:
# All inputs in Python are fields from the genesis input file
# if no value in input file template, default values are filled in
gen.input.keys()

In [ ]:
gen.beam # empty because no beam is loaded 
# you can supply genesis an initial beam distribution.

In [ ]:
gen.param.keys()

# Pick a few parameters each, and find them in the manual.
# Summarize what they are used for. 

Next, look at the lattice info.   

There are two keys in the lattice information: 
- eles  = elements
- param = parameters 

In [ ]:
# list of elements and parameters
# s = end of elements, final positions
# lume-genesis fills in empty spaces for overlap
gen.input['lattice'].keys()

In [ ]:
gen.lattice['param'] # lattice information

Genesis lattice files consist of components in the beamline (from manual):

- AW - Main magnetic field (undulator)
- AD - Drift section
- QF - Quadrupole strength
- QX - Quadrupole offset in x
- QY - Quadrupole offset in y
- SL - Solenoid strength
- CX - Corrector strength in x
- CY - Corrector strength in y


Information included in the element descriptions: 
- strength = magnet strength
- L = length of the structure in measure of the unit length
- d = distance to previous element
- s = distance to the previous element of the same kind

In [ ]:
gen.lattice['eles']

# 4. Output settings

Genesis can output many types of data. There are several flags for data output

- itdp    - time dependant simulation 
- idump   - wavefront output 
- idmpfld - turn on field output
- idmppar - turn on particle distribution otuput
- ippart  - "Write the particle distribution to file at each IPPARTth integration step"
- ipradi  - "Write the field distribution to file at each IPRADIth integration step"

**Note** for list above, parameter = 1 is on, and = 0 is off, unless otherwise noted in manual

In [ ]:
# Dump wavefront output
# set to 0 for no wavefront output
gen.param['idump'] = 1

# Set to 0, for time independant simulation
# This is a good starting point to check if things look reasonable
gen.param['itdp']  = 0

# # Turn on history
gen.param['ippart'] = 10
gen.param['ipradi'] = 10

# Change number of slices 
# gen['nslice'] = 1

# # Turn on/off field output
gen['idmpfld'] = 1

# # Turn on particle output
# gen['idmppar'] = 1

# Change number of particles
# gen['npart'] = 2048

In [ ]:
# Configure the runs with changes made above
gen.configure()
# gen.write_input()

# 5. Run lcls genesis file

In [ ]:
gen.run()

In [ ]:
gen.output['run_info'] # gives info about run time and location

The run above is by default single core.   
You can run also run using mpi on up to 10 cores: 

In [ ]:
!which genesis2-mpi

In [ ]:
# # This command will run the genesis simulation 
# gen.binary_prefixes = ['mpirun', '-n', '2']
# gen.configure()
# gen.run()

In [ ]:
# gen.output['run_info']

# 6. Archiving the data

Saving the data to an h5 file allows you to reload with lume-genesis, and save most data in one file.   
You can reload your data after closing jupyterlab, and don't need to leave the browser window open.

Some notes on output files:  
fld - field history file (can get large)  
par - can get very large if many slices  
dlf (wavefront), dpa (phase space) are final field and particle files

In [ ]:
# Archive data to h5 so that you can reload it later
gen.archive('test_run/lcls_sase.h5')

In [ ]:
#This will only work if 'idumpfld' option is non-zero
gen.write_wavefront('test_run/lcls_sase_wavefront.h5') # save wavefront data to seperate h5 file

# 7. Loading data from archive file & looking at output

In [ ]:
gen.load_archive('test_run/lcls_sase.h5')
# Output data is now saved in gen object

In [ ]:
gen.output.keys() # three categories of saved data

In [ ]:
gen.output['run_info'] #information about simulation run time and location

In [ ]:
gen.output['param'].keys() # input parameters to simulation
# should be similar or same as input params
# some numbers filled in during run with defaults

In [ ]:
# These are the available data types
gen.output['data'].keys() # all the output data

----

# 8. Plotting results

In [ ]:
# Get z values in sim
zlist = gen.output['data']['z'] # 1D array
zlist.shape

In [ ]:
# Get power. This is a 2d array of: slice, z
power = gen.output['data']['power']
power.shape

In [ ]:
plt.plot(zlist, power[0]/1e9);
plt.xlabel('Z (m)'); plt.ylabel('Power (GW)');

In [ ]:
plt.plot(zlist,np.log(power[0]))
plt.xlabel('z along undulator')
plt.ylabel('Power (W)')
plt.show()

In [ ]:
# Use the wavefront file from above
wf  = h5py.File('test_run/lcls_sase_wavefront.h5', 'r') 
dfl = wf['data/000000/meshes/electricField/x']
wf['data/000000/meshes/electricField/x']

In [ ]:
dfl.shape

In [ ]:
# Get parameters from .out file
params = gen.output['param']
my_ncar = params['ncar']
my_dgrid = params['dgrid']

In [ ]:
my_ncar

In [ ]:
my_dgrid

In [ ]:
# Field phase at end, slice 0
def plot_field(dat, dgrid):
    ndat = np.angle(dat)
    plt.imshow(ndat, extent = [1000*dgrid*i for i in [-1,1,-1,1]])
    plt.xlabel('x (mm)')
    plt.ylabel('y (mm)')
    plt.show()

In [ ]:
plot_field(dfl[:, :, 0], my_dgrid )

In [ ]:
# If you run w/ time dependance, you can plot field at multiple points
# plot_field(dfl[:, :, 100], my_dgrid )
# plot_field(dfl[:, :, 200], my_dgrid )
# plot_field(dfl[:, :, 300], my_dgrid )

----

----